In [1]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
import config
from parse_functions import *
# ссылка на основной файл гугл докс (на каждом лите отдельный клиент со ссылками на файлы с данными)
base_link = config.base_link


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [3]:
# забираем данные из основной управленческой таблицы
content = get_data_from_sheet(base_link)
sheet_names = pd.ExcelFile(content)

In [4]:
sheet_names.sheet_names

['Калининград МореАвто(ЮтоКарс)', 'Екатеринбург АтлантикПРО', 'АвтоБелогорье']

In [5]:
for name in sheet_names.sheet_names:
   
    if 'калининград' in name.lower():
        print(name)
        cols_range = 'A:B' # задаем диапазон полей, которые нам нужны
        df = pd.read_excel(content, sheet_name=name, usecols=cols_range, names=['report_name', 'report_link'])
        # нормализуем назания отчетов - приводим в нижний регистр, обрезаем по краям, избаляемся от технических символов(перенос строки и тд)
        df['report_name'] = df['report_name'].str.lower().str.strip().str.replace('\n', ' ')
        # создаем отдельный список с названиями отчетов и проодим по нему через цикл
        reports_list = list(df['report_name'])
        for report in reports_list:
            if 'цели' in report.lower():
                print(report)
                parse_info = df[df['report_name']==report].reset_index(drop=True)

Калининград МореАвто(ЮтоКарс)
цели


In [6]:
google_link = parse_info['report_link'][0]

In [7]:
 # забираем ссылку на гугл докс, из нее получаем ИД и забираем данные 
content = get_data_from_sheet(parse_info['report_link'][0])
# забираем названия листов в гугл доксе
target_sheet_names = pd.ExcelFile(content)

In [8]:
target_sheet_names.sheet_names

['ОП', 'АП', 'Выкуп', 'Парсинг', 'Справочник']

In [16]:
for target_sheet in target_sheet_names.sheet_names:
    if target_sheet.lower() == 'оп':
        print(target_sheet)
        df_tmp = pd.read_excel(content, sheet_name=target_sheet, header=None)
        df_tmp = df_tmp.fillna('')
        df_tmp.columns = df_tmp.iloc[0].str.lower().str.strip().str.replace('\n', ' ') # забираем название полей из файла
        df_tmp = df_tmp.iloc[1:]
        df_tmp = df_tmp[df_tmp['дата начала']!='']
        df_tmp['дата начала'] = pd.to_datetime(df_tmp['дата начала']).dt.date # приводим в формат даты
        df_tmp['дата окончания'] = pd.to_datetime(df_tmp['дата окончания']).dt.date # приводим в формат даты
        df_tmp = df_tmp.reset_index(drop=True)

ОП


In [17]:
df_tmp.head()

,дата начала,дата окончания,название,план
0,2024-09-12,2024-09-18,,
1,2024-09-12,2024-09-18,,
2,2024-09-12,2024-09-18,,
3,2024-09-19,2024-09-25,"Менеджер создал срочность (много интересуются, акция)",0.55
4,2024-09-19,2024-09-25,Менеджер аргументирует возражение клиента,0.7
